## Qlora fine tuning on llama2

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

Installing necessary libraries

In [2]:
from datasets import load_dataset, Dataset, DatasetDict
from dataclasses import dataclass, field
from typing import Optional
import torch
from peft import LoraConfig
from tqdm import tqdm
import pandas as pd
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, AutoTokenizer, pipeline
from trl import SFTTrainer

tqdm.pandas()

# nli_data = load_dataset('klue', 'nli')

# print(nli_data)

# train_data = nli_data['train']
# validation_data = nli_data['validation']


train_data = pd.read_csv('train.csv', sep='\t')
validation_data = pd.read_csv('validation.csv', sep='\t')
dataset_dict = DatasetDict({
    'train': train_data,
    'validation': validation_data
})
train_data = dataset_dict['train']
print(train_data)
validation_data = dataset_dict['validation']
print(validation_data)

                                                   text     label
0     The production is to be liquidated before June...  negative
1     The market making in accordance with the agree...   neutral
2     The largest construction company in Finland , ...   neutral
3     The transaction , which includes US$ 1.5 billi...   neutral
4     MADISON , Wis. , Feb. 6 - PRNewswire - -- Fisk...  positive
...                                                 ...       ...
3401  Based on the first quarter result , existing o...  negative
3402  Aldata said that there are still a number of o...   neutral
3403  The casing comprises a first side casing membe...   neutral
3404  The most significant capital expenditure items...   neutral
3405  Stockmann MasterCard has widened the scope of ...  positive

[3406 rows x 2 columns]
                                                  text     label
0    According to Scanfil , demand for telecommunic...  negative
1    Kemira 's partner in the project is St. Peters..

Arrange the data

In [3]:
import random


# id_to_label = {0:'entailment', 1:'neutral', 2:'contradiction'}

# question_template = "### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. "
# train_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(train_data['premise'],train_data['hypothesis'],train_data['label'])]
# validation_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(validation_data['premise'],validation_data['hypothesis'],validation_data['label'])]

id_to_label = {'negative':0, 'neutral':1, 'positive':2}

question_template = "### Classify the sentiment of the following sentence as negative, neutral or positive."

train_instructions = [f'{question_template}\nStock comments: {x}\n\n### Answer: {y}' for x,y in zip(train_data['text'],train_data['label'])]
validation_instructions = [f'{question_template}\nStock comments: {x}\n\n### Answer: {y}' for x,y in zip(validation_data['text'],validation_data['label'])]


# random.seed(42)
# train_instructions = random.sample(train_instructions, len(train_instructions))
# train_instructions = train_instructions[:300]
# random.seed(42)
# validation_instructions = random.sample(validation_instructions, len(validation_instructions))
# # validation_instructions = validation_instructions[:100]


ds_train = Dataset.from_dict({"text": train_instructions})
ds_validation = Dataset.from_dict({"text": validation_instructions})
instructions_ds_dict = DatasetDict({"train": ds_train, "eval": ds_validation})

In [4]:
print(len(train_instructions))

3406


In [5]:
instructions_ds_dict['train']['text'][0]

'### Classify the sentiment of the following sentence as negative, neutral or positive.\nStock comments: The production is to be liquidated before June 2009 and 325 employees loose their jobs\n\n### Answer: negative'

In [6]:
instructions_ds_dict['eval']['text'][0]

'### Classify the sentiment of the following sentence as negative, neutral or positive.\nStock comments: According to Scanfil , demand for telecommunications network products has fluctuated significantly in the third quarter of 2006 , and the situation is expected to remain unstable for the rest of the year\n\n### Answer: negative'


Training, used beomi/llama-2

In [7]:
model_name = "beomi/llama-2-ko-7b"


@dataclass
class ScriptArguments:
    model_name: Optional[str] = field(default=model_name, metadata={"help": "the model name"})
    dataset_text_field: Optional[str] = field(default="text", metadata={"help": "the text field of the dataset"})
    log_with: Optional[str] = field(default=None, metadata={"help": "use 'wandb' to log with wandb"})
    learning_rate: Optional[float] = field(default=1.41e-5, metadata={"help": "the learning rate"})
    batch_size: Optional[int] = field(default=4, metadata={"help": "the batch size"})
    seq_length: Optional[int] = field(default=512, metadata={"help": "Input sequence length"})
    gradient_accumulation_steps: Optional[int] = field(
        default=2, metadata={"help": "the number of gradient accumulation steps"}
    )
    load_in_8bit: Optional[bool] = field(default=False, metadata={"help": "load the model in 8 bits precision"})
    load_in_4bit: Optional[bool] = field(default=True, metadata={"help": "load the model in 4 bits precision"})
    use_peft: Optional[bool] = field(default=True, metadata={"help": "Wether to use PEFT or not to train adapters"})
    trust_remote_code: Optional[bool] = field(default=True, metadata={"help": "Enable `trust_remote_code`"})
    output_dir: Optional[str] = field(default="output", metadata={"help": "the output directory"})
    peft_lora_r: Optional[int] = field(default=64, metadata={"help": "the r parameter of the LoRA adapters"})
    peft_lora_alpha: Optional[int] = field(default=16, metadata={"help": "the alpha parameter of the LoRA adapters"})
    logging_steps: Optional[int] = field(default=1, metadata={"help": "the number of logging steps"})
    use_auth_token: Optional[bool] = field(default=False, metadata={"help": "Use HF auth token to access the model"})
    num_train_epochs: Optional[int] = field(default=5, metadata={"help": "the number of training epochs"}) #train epoch
    max_steps: Optional[int] = field(default=-1, metadata={"help": "the number of training steps"})
    save_steps: Optional[int] = field(
        default=100, metadata={"help": "Number of updates steps before two checkpoint saves"}
    )
    save_total_limit: Optional[int] = field(default=10, metadata={"help": "Limits total number of checkpoints."})
    push_to_hub: Optional[bool] = field(default=False, metadata={"help": "Push the model to HF Hub"})
    hub_model_id: Optional[str] = field(default=None, metadata={"help": "The name of the model on HF Hub"})


script_args = ScriptArguments()

4-bit quantization implemented

In [8]:
if script_args.load_in_8bit and script_args.load_in_4bit:
    raise ValueError("You can't load the model in 8 bits and 4 bits at the same time")
elif script_args.load_in_8bit or script_args.load_in_4bit:
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=script_args.load_in_8bit, load_in_4bit=script_args.load_in_4bit
    )
    device_map = {"": 0}
    torch_dtype = torch.bfloat16
else:
    device_map = None
    quantization_config = None
    torch_dtype = None

model = AutoModelForCausalLM.from_pretrained(
    script_args.model_name,
    quantization_config=quantization_config,
    device_map=device_map,
    trust_remote_code=script_args.trust_remote_code,
    torch_dtype=torch_dtype,
    use_auth_token=script_args.use_auth_token,
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:465: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

model-00013-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/742M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

Qlora trainer defined

In [9]:
dataset = instructions_ds_dict

training_args = TrainingArguments(
    output_dir=script_args.output_dir,
    per_device_train_batch_size=script_args.batch_size,
    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
    learning_rate=script_args.learning_rate,
    logging_steps=script_args.logging_steps,
    num_train_epochs=script_args.num_train_epochs,
    max_steps=script_args.max_steps,
    report_to=script_args.log_with,
    save_steps=script_args.save_steps,
    save_total_limit=script_args.save_total_limit,
    push_to_hub=script_args.push_to_hub,
    hub_model_id=script_args.hub_model_id,
)

if script_args.use_peft:
    peft_config = LoraConfig(
        r=script_args.peft_lora_r,
        lora_alpha=script_args.peft_lora_alpha,
        bias="none",
        task_type="CAUSAL_LM",
    )
else:
    peft_config = None

trainer = SFTTrainer(
    model=model,
    args=training_args,
    max_seq_length=script_args.seq_length,
    train_dataset=dataset['train'],
    eval_dataset=dataset['eval'],
    dataset_text_field=script_args.dataset_text_field,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)


tokenizer_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3406 [00:00<?, ? examples/s]

Map:   0%|          | 0/378 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Trained on Google colab, 2hr on L4

In [10]:
trainer.train()

trainer.save_model(training_args.output_dir)

Step,Training Loss
1,4.388900
2,4.860300
3,5.140600
4,4.857600
5,4.773500
6,4.762900
7,4.925500
8,4.760100
9,5.036500
10,5.008600


Step,Training Loss
1,4.388900
2,4.860300
3,5.140600
4,4.857600
5,4.773500
6,4.762900
7,4.925500
8,4.760100
9,5.036500
10,5.008600


In [11]:
# trainer.predict(instructions_ds_dict['eval']['text'][1])

Evaluation

In [12]:
model.eval()
# from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, AutoTokenizer, pipeline
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map={'':0},
)

In [13]:
print(instructions_ds_dict['eval']['text'][1])
print(instructions_ds_dict['eval']['text'][1].split('### Answer: ')[0] + '### Answer:')

### Classify the sentiment of the following sentence as negative, neutral or positive.
Stock comments: Kemira 's partner in the project is St. Petersburg Water Works

### Answer: neutral
### Classify the sentiment of the following sentence as negative, neutral or positive.
Stock comments: Kemira 's partner in the project is St. Petersburg Water Works

### Answer:


In [14]:

query = instructions_ds_dict['eval']['text'][1].split('### Answer: ')[0] + '### Answer:'
print(query)
queries = [instructions_ds_dict['eval']['text'][i].split('### Answer: ')[0] + '### Answer:' for i in range(len(instructions_ds_dict['eval']))]
print(queries)
sequences = pipeline(
    queries,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=3,
    early_stopping=True,
    # do_sample=True,
)

### Classify the sentiment of the following sentence as negative, neutral or positive.
Stock comments: Kemira 's partner in the project is St. Petersburg Water Works

### Answer:
['### Classify the sentiment of the following sentence as negative, neutral or positive.\nStock comments: According to Scanfil , demand for telecommunications network products has fluctuated significantly in the third quarter of 2006 , and the situation is expected to remain unstable for the rest of the year\n\n### Answer:', "### Classify the sentiment of the following sentence as negative, neutral or positive.\nStock comments: Kemira 's partner in the project is St. Petersburg Water Works\n\n### Answer:", '### Classify the sentiment of the following sentence as negative, neutral or positive.\nStock comments: Finnish Rautaruukki has been awarded a contract to supply and install steel superstructures for the Partihallsf+rbindelsen bridge in Gothenburg in Sweden\n\n### Answer:', '### Classify the sentiment of th

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:611: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [18]:
results = []

for seq in sequences:
  # print(seq)
  result = seq[0]['generated_text'].split('### Answer:')[1]
  results.append(result)

labels = []
instructions_ds_dict['eval']['text'][0]
print(instructions_ds_dict['eval']['text'][0])
for label in instructions_ds_dict['eval']['text']:
  result = label.split('### Answer:')[1]
  labels.append(result)
print("label is",labels[0])

print("Accuracy: ", (len([1 for x, y in zip(results, labels) if y in x]) / len(labels)))

### Classify the sentiment of the following sentence as negative, neutral or positive.
Stock comments: According to Scanfil , demand for telecommunications network products has fluctuated significantly in the third quarter of 2006 , and the situation is expected to remain unstable for the rest of the year

### Answer: negative
label is  negative
Accuracy:  0.828042328042328


In [ ]:
print(len(instructions_ds_dict['eval']))

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
output_merged_dir = "/content/drive/MyDrive/merged_result/news_classification_llama2_7b/final_merged_checkpoint"

In [21]:

model.save_pretrained( "/content/drive/MyDrive/merged_result/news_classification_llama2_7b/final_merged_checkpoint")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2577: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 4096}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [22]:
tokenizer.save_pretrained(output_merged_dir)

('/content/drive/MyDrive/merged_result/news_classification_llama2_7b/final_merged_checkpoint/tokenizer_config.json',
 '/content/drive/MyDrive/merged_result/news_classification_llama2_7b/final_merged_checkpoint/special_tokens_map.json',
 '/content/drive/MyDrive/merged_result/news_classification_llama2_7b/final_merged_checkpoint/tokenizer.json')

In [26]:
!git clone https://github.com/AI4Finance-Foundation/FinNLP.git

batch_size = 8
import sys
sys.path.append('/content/FinNLP/')
from finnlp.benchmarks.fiqa import test_fiqa , add_instructions
from finnlp.benchmarks.fpb import test_fpb
from finnlp.benchmarks.fiqa import test_fiqa , add_instructions
from finnlp.benchmarks.tfns import test_tfns
from finnlp.benchmarks.nwgi import test_nwgi

fatal: destination path 'FinNLP' already exists and is not an empty directory.


In [27]:
res = test_fiqa(model, tokenizer, prompt_fun = add_instructions, batch_size = batch_size)



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: This $BBBY stock options trade would have more than doubled your money https://t.co/Oa0loiRIJL via @TheStreet
Answer: 


Total len: 275. Batchsize: 8. Total steps: 35


  0%|          | 0/35 [00:38<?, ?it/s]


KeyboardInterrupt: 

In [28]:
res = test_fpb(model, tokenizer, batch_size = batch_size)

The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: L&T has also made a commitment to redeem the remaining shares by the end of 2011 .
Answer: 


Total len: 1212. Batchsize: 8. Total steps: 152


  0%|          | 0/152 [02:39<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
res = test_tfns(model, tokenizer, batch_size = batch_size)

In [ ]:
res = test_nwgi(model, tokenizer, batch_size = batch_size)

In [ ]:
res = test_fiqa(model, tokenizer, prompt_fun = add_instructions, batch_size = batch_size)

In [ ]:
# nli_data = load_dataset('klue', 'nli')

# print(nli_data)

# train_data = nli_data['train']
# validation_data = nli_data['validation']
# id_to_label = {0:'entailment', 1:'neutral', 2:'contradiction'}

# question_template = "### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. "
# train_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(train_data['premise'],train_data['hypothesis'],train_data['label'])]
# validation_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(validation_data['premise'],validation_data['hypothesis'],validation_data['label'])]
# ds_train = Dataset.from_dict({"text": train_instructions})
# ds_validation = Dataset.from_dict({"text": validation_instructions})
# instructions_ds_dict = DatasetDict({"train": ds_train, "eval": ds_validation})
# print(instructions_ds_dict['eval']['text'][1])
# print(instructions_ds_dict['eval']['text'][1].split('### Assistant: ')[0] + '### Assistant:')

In [ ]:
query = instructions_ds_dict['eval']['text'][1].split('### Assistant: ')[0] + '### Assistant:'
print(query)
queries = [instructions_ds_dict['eval']['text'][i].split('### Assistant: ')[0] + '### Assistant:' for i in range(len(instructions_ds_dict['eval']))]
print(queries)
print("queries[0] is",queries[0])
sequences = pipeline(
    queries,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=3,
    early_stopping=True,
    # do_sample=True,
)


# query = instructions_ds_dict['eval']['text'][1].split('### Answer: ')[0] + '### Answer:'
# print(query)
# queries = [instructions_ds_dict['eval']['text'][i].split('### Answer: ')[0] + '### Answer:' for i in range(len(instructions_ds_dict['eval']))]
# print(queries)
# sequences = pipeline(
#     queries,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
#     max_new_tokens=3,
#     early_stopping=True,
#     # do_sample=True,
# )

### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. 
premise: 그는 플루타르코스의 저서에 끼어있던 아내에게 보내는 편지를 번역하였는데 여기서 플루타르코스는 2년 연상의 언니를 잃고 비탄에 빠진 아내를 위한하고 있다.
hypothesis: 플루타르코스의 처형이 죽었다.

### Assistant:
['### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. \npremise: 출시 후 3개월 간은 오픈 베타 서비스 기간이다.\nhypothesis: 출시 후 3개월 간은 오픈 베타 서비스를 받을 수 있다.\n\n### Assistant:', '### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. \npremise: 그는 플루타르코스의 저서에 끼어있던 아내에게 보내는 편지를 번역하였는데 여기서 플루타르코스는 2년 연상의 언니를 잃고 비탄에 빠진 아내를 위한하고 있다.\nhypothesis: 플루타르코스의 처형이 죽었다.\n\n### Assistant:', '### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. \npremise: 2600억원 규모 무역보증을 운영하고 디지털 기술과의 접목 등 새로운 정책도 도입한다.\nhypothesis: 새로운 정책을 도입할 것이다.\n\n### Assistant:', '### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. \npremise: 법률자문은 1인당 최대 100만 원, 변호사비 1인당 최대 200만 원, 소송비용은 인지대 등 실비지원한다.\nhypothesis: 1인당 최대 400만원까지 변호사비를 지원한다.\n\n### Assistant:', '### H

In [ ]:
results = []

for seq in sequences:
  # print(seq)
  result = seq[0]['generated_text'].split('### Assistant:')[1]
  results.append(result)

labels = []
instructions_ds_dict['eval']['text'][0]
print(instructions_ds_dict['eval']['text'][0])
for label in instructions_ds_dict['eval']['text']:
  result = label.split('### Assistant:')[1]
  labels.append(result)

print("Accuracy: ", (len([1 for x, y in zip(results, labels) if y in x]) / len(labels)))


# results = []

# for seq in sequences:
#   # print(seq)
#   result = seq[0]['generated_text'].split('### Answer:')[1]
#   results.append(result)

# labels = []
# instructions_ds_dict['eval']['text'][0]
# print(instructions_ds_dict['eval']['text'][0])
# for label in instructions_ds_dict['eval']['text']:
#   result = label.split('### Answer:')[1]
#   labels.append(result)

# print("Accuracy: ", (len([1 for x, y in zip(results, labels) if y in x]) / len(labels)))

### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. 
premise: 출시 후 3개월 간은 오픈 베타 서비스 기간이다.
hypothesis: 출시 후 3개월 간은 오픈 베타 서비스를 받을 수 있다.

### Assistant: entailment
Accuracy:  1.0
